In [9]:
import requests
import json
from bs4 import BeautifulSoup
from typing import List
import time
from concurrent.futures import ThreadPoolExecutor
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def extract_urls_lists(search_page_url: str) -> List[str]:
    response = requests.get(search_page_url, params = {})
    soup = BeautifulSoup(response.text, 'html.parser')

    list_of_urls=[]

    for article in soup.find_all('iw-search-card-rendered'):
        for link in article.find_all('a'):
            list_of_urls.append(link.get('href'))
    return list_of_urls

def extract_x_urls(number_of_urls: int=30) -> List[str]:
    immo_base_url = "https://www.immoweb.be/fr/recherche/maison/a-vendre?countries=BE&page="
    immo_end_url = "&orderBy=relevance"
    searchpages_urls_list =[]

    for index in range(1,round(min(334,number_of_urls/30 + 1))):
        full_url = immo_base_url + str(index) + immo_end_url
        searchpages_urls_list.append(full_url)
    
    full_urls_list =[]

    with ThreadPoolExecutor() as pool:
        full_urls_list=list(pool.map(extract_urls_lists, searchpages_urls_list))  

    if number_of_urls>9990:
        immo_base_url = "https://www.immoweb.be/fr/recherche/appartement/a-vendre?countries=BE&page="
        immo_end_url = "&orderBy=relevance"
        searchpages_urls_list =[]

        for index in range(1,round(min(334,(number_of_urls-9990)/30 + 1))):
            full_url = immo_base_url + str(index) + immo_end_url
            searchpages_urls_list.append(full_url)

        with ThreadPoolExecutor() as pool:
            full_urls_list+=list(pool.map(extract_urls_lists, searchpages_urls_list))  
            
    return list(set(sum(full_urls_list,[])))



14999


In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
pd.set_option('display.width', None)
def make_one_data_frame(url):
  """
  session = requests.Session()
  retry = Retry(connect=3, backoff_factor=0.5)
  adapter = HTTPAdapter(max_retries=retry)
  session.mount('http://', adapter)
  session.mount('https://', adapter)
  """
  response = requests.get(url)
  test_page = response.text

  dfs = pd.read_html(test_page)

  full_df = pd.concat(dfs).dropna(thresh=2).T
  full_df.columns = full_df.iloc[0]
  full_df = full_df[1:]
  full_df = full_df.loc[:, full_df.columns.isin(['Neighbourhood or locality','Building condition','Number of frontages','Living area','Kitchen type','Bedrooms','Bathrooms','Furnished','Surface of the plot','Garden surface','Swimming pool','Price','Terrace'])]
  return full_df
  # pd.read_html reads in all tables and returns a list of DataFrames



In [3]:

list_of_urls = extract_x_urls(90)
print('list ready')

with ThreadPoolExecutor() as pool:
        full_df= pd.concat(pool.map(make_one_data_frame, list_of_urls))    

print('done extracting')


#full_df = pd.DataFrame.from_dict(map(dict,dataframes_list))
full_df

list ready


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
list_of_urls = extract_x_urls(30)
print
with ThreadPoolExecutor() as pool:

    list_of_df= list(pool.map(make_one_data_frame, list_of_urls))

df3= pd.concat(list_of_df, ignore_index=True)

df3

,À partir de,Année de construction,État du bâtiment,Largeur de la façade à rue,Nombre de façades,Type d’environnement,Surface habitable,Surface du salon,Salle à manger,Type de cuisine,...,Jardin,Orientation du jardin,Terrasse,Interphone vidéo,Visite virtuelle,Charges mensuelles,Obligation de construire,Panneaux solaires,Attestation de conformité des citernes à mazout,Parlophone
0,Après la signature de l’acte,1960,Bon,4 m,2,Isolé,133 m² mètres carrés,16 m² mètres carrés,Oui,Pas équipée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Après la signature de l’acte,2010,Bon,7.5 m,2,Urbain,160 m² mètres carrés,NaN,NaN,Équipée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Après la signature de l’acte,2012,Excellent état,12 m,3,Isolé,175 m² mètres carrés,40 m² mètres carrés,NaN,Américaine équipée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Après la signature de l’acte,1958,À rénover,9 m,2,Campagne,240 m² mètres carrés,31 m² mètres carrés,NaN,Semi-équipée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Après la signature de l’acte,1960,Bon,14 m,4,Isolé,355 m² mètres carrés,38 m² mètres carrés,Oui,Américaine équipée,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,1909,À rafraîchir,10 m,2,NaN,282 m² mètres carrés,52 m² mètres carrés,NaN,Équipée,...,Oui,Est,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Après la signature de l’acte,1950,Bon,10.5 m,4,Urbain,450 m² mètres carrés,50 m² mètres carrés,NaN,Équipée,...,NaN,NaN,Oui,Oui,NaN,NaN,NaN,NaN,NaN,NaN
7,Après la signature de l’acte,1899,À rénover,4 m,3,NaN,123 m² mètres carrés,17 m² mètres carrés,Oui,Pas équipée,...,NaN,NaN,NaN,NaN,Visite virtuelle,NaN,NaN,NaN,NaN,NaN
8,NaN,1899,Fraîchement rénové,5 m,3,Urbain,61 m² mètres carrés,NaN,NaN,Équipée,...,NaN,NaN,NaN,NaN,NaN,17 € 17 €,NaN,NaN,NaN,NaN
9,NaN,1978,Bon,6 m,2,Urbain,95 m² mètres carrés,42 m² mètres carrés,NaN,Équipée,...,NaN,NaN,NaN,NaN,NaN,230 € 230 €,NaN,NaN,NaN,NaN
